Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [3]:
print(df.value_counts('gender'))
print(df.value_counts('gender',normalize=True))

gender
Male      3555
Female    3488
dtype: int64
gender
Male      0.504756
Female    0.495244
dtype: float64


##### 2. Какое количество уникальных значений у поля InternetService?

In [4]:
print(df.value_counts('InternetService'))
print(f'Уникальных значений {len(df.value_counts("InternetService"))}')
df['InternetService'].unique(), df['InternetService'].nunique()

InternetService
Fiber optic    3096
DSL            2421
No             1526
dtype: int64
Уникальных значений 3


(array(['DSL', 'Fiber optic', 'No'], dtype=object), 3)

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [5]:
df[df['TotalCharges']!=" "]['TotalCharges'].astype(float).describe()[['50%','mean','std']]
    

50%     1397.475000
mean    2283.300441
std     2266.771362
Name: TotalCharges, dtype: float64

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)


Поскольку в данных присутсвовали значения пустой строки - " ", то пандас считал их типом объект и не мог посчитать статистики.

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [6]:
df['PhoneService'] = df['PhoneService'].map({'Yes':1, 'No':0})

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [7]:
df.loc[df['TotalCharges'] ==' ','TotalCharges'] = np.nan

In [8]:
df['TotalCharges'].fillna(0,inplace=True)

In [9]:
df['TotalCharges'] = df['TotalCharges'].astype(np.float32)

In [10]:
df['TotalCharges'].describe()[['50%','mean','std']]

50%     1394.550049
mean    2279.732178
std     2266.794434
Name: TotalCharges, dtype: float64

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [11]:
df['Churn'] = df['Churn'].map({'Yes':1, 'No':0})

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [12]:
for i in ['StreamingMovies', 'StreamingTV', 'TechSupport' ]:
    df[i] = df[i].map({'Yes':1, 'No':0})

##### 8. Заполните пропуски в поле PhoneService значением 0

In [13]:
print(df['PhoneService'].value_counts())
print(len(df))
sum(df['PhoneService'].value_counts())

1    6361
0     682
Name: PhoneService, dtype: int64
7043


7043

PhoneService - Нет пропусков

Наврное имелось ввиду заполнить пропуски в ['StreamingMovies', 'StreamingTV', 'TechSupport' ]

In [14]:
c = pd.Index(['StreamingMovies', 'StreamingTV', 'TechSupport' ])

In [15]:
df[c] = df[c].fillna(0)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [16]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
df = df[columns]
df.head(3)



,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.850000,0.0,0.0,0.0,0
1,Male,34,1,1889.500000,0.0,0.0,0.0,0
2,Male,2,1,108.150002,0.0,0.0,0.0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [17]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
X_train,X_test, y_train, y_test = train_test_split (df.drop(target,axis=1),df[target],test_size = 0.2,random_state = 2, shuffle = True)


In [18]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((5634, 7), (1409, 7), (5634,), (1409,))

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [20]:
#Ваш код здесь
gender.fit_transform(df)

,gender_Female,gender_Male
0,1,0
1,0,1
2,0,1
3,0,1
4,1,0
...,...,...
7038,0,1
7039,1,0
7040,1,0
7041,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [21]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

Нормирование полезно для определенных моделей, например линейных, к ближайших соседей.

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [22]:
#Ваш код здесь
total_charges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])


Объединение всех "кубиков" очень легко сделать таким образом

In [23]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [24]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', total_charges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [25]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feature_processing),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 Pipeline(steps=[('feats',
                                  FeatureUnion(transformer_list=[('tenure',
                                                                  Pipeline(steps=[('selector',
                                                                                   NumberSelector(key='tenure')),
                                                                                  ('standard',
                                                                                   StandardScaler())])),
                                                                 ('TotalCharges',
                                                                  Pipeline(steps=[('selector',
                                                                                   NumberSelector(key='TotalCharges')),
                                                                                  ('standard',
                                                   

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [26]:
#Ваш код здесь
preds = pipeline.predict_proba(X_test)

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [27]:
from sklearn.metrics import roc_auc_score, log_loss
roc_auc_score(y_test,preds[:,1]),log_loss(y_test,preds)



(0.7772460376786159, 0.8207197564086318)

### Сохраним наш пайплайн

In [30]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)

In [29]:
conda install dill


Note: you may need to restart the kernel to use updated packages.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - dill


The following NEW packages will be INSTALLED:

  dill               pkgs/main/noarch::dill-0.3.3-pyhd3eb1b0_0

The following packages will be UPDATED:

  conda              conda-forge::conda-4.9.0-py38h9bdc248~ --> pkgs/main::conda-4.9.2-py38haa95532_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
